In [46]:
import pandas as pd
import numpy as np
import datetime
from nvd3 import lineChart
from pprint import pprint as pp
import os
from timeit import Timer
import random

df = pd.read_csv('../data/all_but_champ/match_log.csv')

df.head()








,match-id,name,match-scores-csv,completed-at,match-updated-at,player1-name,player2-name,player1-score,player2-score,match-length,player1_ELO,player2_ELO,player1_ELO_change,player2_ELO_change,tournament_type
0,27383858,Spur or the Moment 10/22/14,2-3,2014-10-30 15:57:59-06:00,2014-10-29 16:55:21-06:00,Ahmed Abdelgwad,Matic Kovac,2.0,3.0,3.0,1466.063465,1533.936535,-16.616027,16.616027,Spur of the moment
1,27383858,Spur or the Moment 10/22/14,2-3,2014-10-30 22:57:59+01:00,2014-10-29 23:55:21+01:00,Ahmed Abdelgwad,Matic Kovac,2.0,3.0,3.0,1450.092270,1549.907730,-15.971195,15.971195,Spur of the moment
2,27383859,Spur or the Moment 10/22/14,2-3,2014-10-30 15:57:59-06:00,2014-10-30 15:57:51-06:00,Blanche Galietta,Matic Kovac,2.0,3.0,3.0,1483.525019,1566.086161,-16.474981,16.178431,Spur of the moment
3,27383859,Spur or the Moment 10/22/14,2-3,2014-10-30 22:57:59+01:00,2014-10-30 22:57:51+01:00,Blanche Galietta,Matic Kovac,2.0,3.0,3.0,1467.729950,1581.595203,-15.795070,15.509042,Spur of the moment
4,27681579,Great Pumpkin Tournament,5-8,2014-11-09 14:04:36-05:00,2014-10-31 13:37:33-04:00,Paige Wisley Mulick,Heather Rose,5.0,8.0,8.0,1471.715729,1528.284271,-28.284271,28.284271,general


/Users/jacob/anaconda/envs/abg_stats/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jacob/anaconda/envs/abg_stats/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jacob/anaconda/envs/abg_stats/lib/python3.5/site-packages/pandas/core/generic.py:4485: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_